In [33]:
def get_features(dataframe, 
                column, 
                links = [],
                danceability = [],
                energy = [],
                loudness = [],
                speechiness = [],
                acousticness = [],
                instrumentalness = [],
                liveness = [],
                valence = [],
                tempo = []):

    for link in dataframe[column]:  
        
        connection = sp.audio_features(link)[0]
        
        if connection is not None:
            links.append(link)
            danceability.append(connection.get("danceability", None))
            energy.append(connection.get("energy", None))
            loudness.append(connection.get("loudness", None))
            speechiness.append(connection.get("speechiness", None))
            acousticness.append(connection.get("acousticness", None))
            instrumentalness.append(connection.get("instrumentalness", None))
            liveness.append(connection.get("liveness", None))
            valence.append(connection.get("valence", None))
            tempo.append(connection.get("tempo", None))
        else:
            links.append(None)
            danceability.append(None)
            energy.append(None)
            loudness.append(None)
            speechiness.append(None)
            acousticness.append(None)
            instrumentalness.append(None)
            liveness.append(None)
            valence.append(None)
            tempo.append(None)

  
    features_df = pd.DataFrame({"song_url": links,
                                "danceability": danceability,
                                "energy": energy,
                                "loudness": loudness,
                                "speechiness": speechiness,
                                "acousticness": acousticness,
                                "instrumentalness": instrumentalness,
                                "liveness": liveness,
                                "valence": valence,
                                "tempo": tempo})
    return features_df

In [34]:
import sqlite3
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import sys
sys.path.append("../notebooks/")
from spot_secrets import cid, secret

# Connect to the database
conn = sqlite3.connect("spotify_db.db")

# Create a cursor
cursor = conn.cursor()

# Check favorite artist
user_id = "8d93156f-1108-45b4-9b25-4565b14faec2"
query = "SELECT end_time, artist_name, song_name, ms_played FROM Streaming_data WHERE UserID == '{}'".format(user_id)
cursor.execute(query)
data = cursor.fetchall()
data_df = pd.DataFrame(data, columns=["end_time", "artist_name", "song_name", "ms_played"])
data_df = data_df[data_df["end_time"].str.startswith("2022")]
data_df = data_df.drop_duplicates(subset=['song_name'])
data_df = data_df.head(int(len(data_df) * 0.5))
data_df.head()

,end_time,artist_name,song_name,ms_played
3942,2022-01-01 20:00,Sloppy Jane,Madison,226004
3943,2022-01-01 20:02,"Tyler, The Creator",SIR BAUDELAIRE (feat. DJ Drama),88624
3944,2022-01-01 20:04,"Tyler, The Creator",CORSO,146481
3945,2022-01-01 20:06,"Tyler, The Creator",LEMONHEAD (feat. 42 Dugg),130980
3946,2022-01-01 20:08,"Tyler, The Creator",WUSYANAME (feat. Youngboy Never Broke Again & ...,121572


In [35]:
#Authentication
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [36]:
# download links of songs
songs_links = []
for index, song in data_df.iterrows():
    results = sp.search(q='artist:' + data_df["artist_name"][index] + " track:" + data_df["song_name"][index], type='track')
    items = results['tracks']['items']

    if len(items) != 0:
        try:
            link = items[1]['href']
        except:
            link = items[0]['href']
    else:
        link = ""
    songs_links.append(link)

In [37]:
data_df['song_url'] = songs_links
data_df = data_df[data_df['song_url'] != ""] 

In [38]:
features = get_features(data_df, "song_url")

Expected id of type track but found type tracks https://api.spotify.com/v1/tracks/2kZWHquuar29ItjiV0XXCe
Expected id of type track but found type tracks https://api.spotify.com/v1/tracks/4ydmav4vl7hebadtfZtMrg
Expected id of type track but found type tracks https://api.spotify.com/v1/tracks/46oHvXwpEZOFX518we1nJD
Expected id of type track but found type tracks https://api.spotify.com/v1/tracks/5fbHRCsGpFIOLRtlhCRFDR
Expected id of type track but found type tracks https://api.spotify.com/v1/tracks/5B0kgjHULYJhAQkK5XsMoC
Expected id of type track but found type tracks https://api.spotify.com/v1/tracks/64v1g2HcPumBz2Wd1rT56b
Expected id of type track but found type tracks https://api.spotify.com/v1/tracks/3JJL91ilRV6fXhKqu0FGXs
Expected id of type track but found type tracks https://api.spotify.com/v1/tracks/2goHCrsDJaflEwQ930ef0o
Expected id of type track but found type tracks https://api.spotify.com/v1/tracks/13HsOwrwTA5HgLuoaDHncP
Expected id of type track but found type tracks https:/

In [39]:
data_df.head()

,end_time,artist_name,song_name,ms_played,song_url
3942,2022-01-01 20:00,Sloppy Jane,Madison,226004,https://api.spotify.com/v1/tracks/2kZWHquuar29...
3943,2022-01-01 20:02,"Tyler, The Creator",SIR BAUDELAIRE (feat. DJ Drama),88624,https://api.spotify.com/v1/tracks/4ydmav4vl7he...
3944,2022-01-01 20:04,"Tyler, The Creator",CORSO,146481,https://api.spotify.com/v1/tracks/46oHvXwpEZOF...
3945,2022-01-01 20:06,"Tyler, The Creator",LEMONHEAD (feat. 42 Dugg),130980,https://api.spotify.com/v1/tracks/5fbHRCsGpFIO...
3946,2022-01-01 20:08,"Tyler, The Creator",WUSYANAME (feat. Youngboy Never Broke Again & ...,121572,https://api.spotify.com/v1/tracks/5B0kgjHULYJh...


In [40]:
features = features.dropna()
data_df = data_df.merge(features, on="song_url")
data_df = data_df.drop(["end_time", "song_url", "ms_played"], axis=1)

In [41]:
data_df

,artist_name,song_name,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Sloppy Jane,Madison,0.305,0.277,-11.142,0.0337,0.686,0.000203,0.2960,0.0772,127.377
1,"Tyler, The Creator",SIR BAUDELAIRE (feat. DJ Drama),0.467,0.635,-6.115,0.1540,0.314,0.000004,0.8440,0.5100,127.596
2,"Tyler, The Creator",CORSO,0.703,0.845,-4.851,0.3370,0.169,0.000000,0.1220,0.7450,97.672
3,"Tyler, The Creator",LEMONHEAD (feat. 42 Dugg),0.620,0.586,-6.612,0.2120,0.155,0.000000,0.5350,0.4870,137.131
4,"Tyler, The Creator",WUSYANAME (feat. Youngboy Never Broke Again & ...,0.493,0.653,-5.666,0.0466,0.269,0.000000,0.5110,0.8520,142.347
...,...,...,...,...,...,...,...,...,...,...,...
1492,Angèle,Balance ton quoi,0.703,0.442,-10.383,0.0823,0.594,0.000024,0.2760,0.3810,115.934
1493,Lomepal,Yeux disent,0.741,0.631,-7.133,0.1680,0.545,0.000033,0.1710,0.3660,113.710
1494,Sopico,Unplugged #4: La nuit,0.578,0.343,-14.614,0.2200,0.786,0.000003,0.1100,0.4730,102.476
1495,Jan-Rapowanie,boję się,0.638,0.561,-12.179,0.0494,0.515,0.037200,0.0895,0.5330,169.982


In [42]:
user_id = "30102b2b-262f-4e71-8bbe-c12b565e6641"
conn = sqlite3.connect("spotify_db.db")

In [43]:

for i, row in data_df.iterrows():
    conn.execute("INSERT INTO User_songs_info VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", [user_id,
                                                                                  row["artist_name"],
                                                                                  row["song_name"],
                                                                                  row["danceability"],
                                                                                  row["energy"],
                                                                                  row["loudness"],
                                                                                  row["speechiness"],
                                                                                  row["acousticness"],
                                                                                  row["instrumentalness"],
                                                                                  row["liveness"],
                                                                                  row["valence"],
                                                                                  row["tempo"],
    ])

    conn.commit()